# PDIOT ML

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
path = "/content/drive/MyDrive/Colab Notebooks/pdiot-ml"
os.chdir(path)

task_index = 1 # action
# task_index = 21 # stationary action
# task_index = 221 # 22x repiratory type
# task_index = 31 # stationary action
# task_index = 32 # repiratory type and other
# task_index = 4 # increase case 32 with gyro and accl
# task_index = 5 # one model for all

# sensor = 'thingy'
sensor = 'respeck'

# tainable = False
tainable = True

window_size = 50  # Define the size of the window
stride = 1  # Define the stride of the window

match task_index:
    case 4:
        # norm the gyro data
        tag = f'accl_gyro_norm'
    case 5:
        # norm the gyro data
        tag = "accl_gyro_norm"
    case _:
        tag = f'accl_only_no_norm'

model_path = f'model/model_{sensor}_{tag}_task_{task_index}_{window_size}.h5'
data_folder = "pdiot-data/updated_anonymized_dataset_2023/Respeck"


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import concurrent.futures

# Read multiple CSV files and label them
# data_folder = "pdiot-data/updated_anonymized_dataset_2023/Thingy"
files = []
for folder_name in os.listdir(data_folder):
    if folder_name == ".gitkeep":
        continue
    files += [(os.path.join(data_folder, folder_name), file_name) for file_name in os.listdir(os.path.join(data_folder, folder_name))]

data_list = []
labels = []

label_action_dict = {
    "ascending": "ascending stairs",
    "descending": "descending stairs",
    "lyingRight": "lying down on right",
    "lyingLeft": "lying down on left",
    "lyingBack": "lying down on back",
    "lyingStomach": "lying down on stomach",
    "miscMovement": "miscellaneous movements",
    "shuffleWalking": "shuffle walking",
    "normalWalking": "normal walking",
    "sitting": "sitting/standing",
    "standing": "sitting/standing"
}

label_type_dict = {
    "breathingNormal": "normal",
}


def process_file(file_path, file_name, task_index, label_action_dict, label_type_dict):
    if file_name.endswith('.csv'):
        df = pd.read_csv(os.path.join(file_path, file_name))

        label_action, label_type = file_name.replace('.csv', '').split("_")[2:4]
        label_action = label_action_dict[label_action] if label_action in label_action_dict.keys() else label_action
        label_type = label_type_dict[label_type] if label_type in label_type_dict.keys() else label_type

        match task_index:
            case 1:
                if label_type not in ["normal"]:
                    return None, None
                label = label_action
            case 21:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_action
            case 221:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_action != "sitting/standing":
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_type
            case 222:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_action != "lying down on left":
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_type
            case 223:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_action != "lying down on back":
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_type
            case 224:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_action != "lying down on stomach":
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_type
            case 225:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_action != "lying down on right":
                    return None, None
                if label_type not in ["normal", "coughing", "hyperventilating"]:
                    return None, None
                label = label_type
            case 31:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_type in ["talking", "eating", "singing", "laughing"]:
                    label_type = "other"
                label = label_action
            case 32:
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_type in ["talking", "eating", "singing", "laughing"]:
                    label_type = "other"
                label = label_type
            case 4:
                # increase case 3
                if label_action in ["normal walking", "ascending stairs", "descending stairs", "shuffle walking", "running", "miscellaneous movements"]:
                    return None, None
                if label_type in ["talking", "eating", "singing", "laughing"]:
                    label_type = "other"
                label = " ".join([label_action, label_type])

            case 5:
                # if label_type in ["coughing", "hyperventilating"]:
                #     label_type = "abnormal"
                # if label_type in ["talking", "eating", "singing", "laughing"]:
                #     label_type = "other"
                label = " ".join([label_action, label_type])

        match task_index:
            case 4:
                data = df[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
            case 5:
                data = df[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
            case _:
                data = df[['accel_x', 'accel_y', 'accel_z']]

        return label, data

    return None, None

In [4]:
# Using ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_file = {executor.submit(process_file, file_path, file_name, task_index, label_action_dict, label_type_dict): (file_path, file_name) for file_path, file_name in files}

    for future in concurrent.futures.as_completed(future_to_file):
        file_path, file_name = future_to_file[future]
        try:
            label, data = future.result()
            if label is not None and data is not None:
                labels.append(label)
                data_list.append(data)
        except Exception as exc:
            print(f'{file_path}/{file_name} generated an exception: {exc}')


# # Stand the accel data
# tag += "_stand"
# for i in range(len(data_list)):
#     for col in ['accel_x', 'accel_y', 'accel_z']:
#         data_list[i][col] = (data_list[i][col] - data_list[i][col].mean()) / data_list[i][col].std()


match task_index:
    case 4:
        # norm the gyro data
        for i in range(len(data_list)):
            for col in ['gyro_x', 'gyro_y', 'gyro_z']:
                # https://pdf1.alldatasheet.com/datasheet-pdf/view/678850/AD/ADXRS300_15.html
                min_val = -300
                max_val = 300

                data_list[i][col] = (data_list[i][col] - min_val) / (max_val - min_val)
    case 5:
        # norm the gyro data
        for i in range(len(data_list)):
            for col in ['gyro_x', 'gyro_y', 'gyro_z']:
                # https://pdf1.alldatasheet.com/datasheet-pdf/view/678850/AD/ADXRS300_15.html
                min_val = -300
                max_val = 300

                data_list[i][col] = (data_list[i][col] - min_val) / (max_val - min_val)


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.signal import spectrogram

import pandas as pd
import numpy as np
import random

import random

def augment_data(data_list, labels):
    augmented_data = []
    augmented_labels = []

    for index, data in enumerate(data_list):
        # Add original data and label
        augmented_data.append(data)
        augmented_labels.append(labels[index])

        # # Add a random number to each column for augmentation
        # augmented_data_item = data.copy()
        # for column in augmented_data_item.columns:
        #     mean_val = augmented_data_item[column].mean()
        #     deviation = mean_val * 0.01  # 1% of mean
        #     random_shift = random.uniform(-deviation, deviation)
        #     augmented_data_item[column] += random_shift

        # # Add augmented data and label
        # augmented_data.append(augmented_data_item)
        # augmented_labels.append(labels[index])  # Same label for augmented data

    return augmented_data, augmented_labels

# Split the augmented data list and labels into training and test sets
data_train, data_temp, labels_train, labels_temp = train_test_split(
    data_list, labels, test_size=0.3, random_state=42)

data_test, data_val, labels_test, labels_val = train_test_split(
    data_temp, labels_temp, test_size=0.3, random_state=42)

# Apply data augmentation
data_train, labels_train = augment_data(data_train, labels_train)


In [6]:
def compute_spectrograms_and_augment_single(sequence, label, window_size, stride, sampling_times):
    X_spectrograms = []
    y_spectrograms = []

    # Original data processing
    for j in range(0, len(sequence) - window_size, stride):
        window = sequence.iloc[j:j + window_size]
        X_spectrograms.append(window)
        y_spectrograms.append(label)

    # Data augmentation
    for _ in range(sampling_times):
        # Select a random sampling ratio between 0.8 and 1.2
        sampling_ratio = random.uniform(0.8, 1.2)
        new_length = int(len(sequence) * sampling_ratio)

        # Handle DataFrame sequence
        augmented_sequence = pd.DataFrame(index=np.linspace(0, len(sequence) - 1, new_length))
        for column in sequence.columns:
            interpolated = np.interp(
                augmented_sequence.index,
                sequence.index,
                sequence[column]
            )
            augmented_sequence[column] = interpolated


        # # Calculate new indices for linear interpolation
        # new_indices = np.linspace(0, len(sequence) - 1, new_length)

        # # Initialize augmented sequence DataFrame
        # augmented_sequence = pd.DataFrame(index=new_indices)

        # # Efficient linear interpolation for each column
        # for column in sequence.columns:
        #     y_values = sequence[column].values
        #     x_values = np.arange(len(y_values))

        #     # Calculate slopes for linear interpolation
        #     slopes = (y_values[1:] - y_values[:-1]) / (x_values[1:] - x_values[:-1])

        #     # Find the index positions in the original data for each new index
        #     idx_pos = np.searchsorted(x_values[1:], new_indices)

        #     # Calculate the interpolated values
        #     interpolated_values = y_values[idx_pos] + slopes[idx_pos] * (new_indices - x_values[idx_pos])

        #     augmented_sequence[column] = interpolated_values

        # Process the augmented sequence
        for j in range(0, len(augmented_sequence) - window_size, stride):
            window = augmented_sequence.iloc[j:j + window_size]
            X_spectrograms.append(window)
            y_spectrograms.append(label)

    return X_spectrograms, y_spectrograms

import concurrent.futures

def process_data_in_parallel(data, labels, window_size, stride, sampling_times):
    X_spectrograms_combined = []
    y_spectrograms_combined = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(compute_spectrograms_and_augment_single, sequence, label, window_size, stride, sampling_times) for sequence, label in zip(data, labels)]

        for future in concurrent.futures.as_completed(futures):
            try:
                X_spectrograms, y_spectrograms = future.result()
                X_spectrograms_combined.extend(X_spectrograms)
                y_spectrograms_combined.extend(y_spectrograms)
            except Exception as exc:
                print(f'An exception occurred: {exc}')

    return np.array(X_spectrograms_combined), np.array(y_spectrograms_combined)

# Process training and test data in parallel
X_train, y_train = process_data_in_parallel(data_train, labels_train, window_size, stride, 0)
X_test, y_test = process_data_in_parallel(data_test, labels_test, window_size, stride, 0)
X_val, y_val = process_data_in_parallel(data_val, labels_val, window_size, stride, 0)

In [7]:
# Label encode and one-hot encode labels
label_encoder = LabelEncoder()
integer_encoded_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(integer_encoded_train)

integer_encoded_test = label_encoder.fit_transform(y_test)
y_test = to_categorical(integer_encoded_test)

integer_encoded_val = label_encoder.fit_transform(y_val)
y_val = to_categorical(integer_encoded_val)

integer_mapping = {l: i for i, l in enumerate(label_encoder.classes_)}

integer_mapping = {i: l for i, l in enumerate(label_encoder.classes_)}

result = False
match task_index:
    case 1:
        result = integer_mapping == {
            0: 'ascending stairs',
            1: 'descending stairs',
            2: 'lying down on back',
            3: 'lying down on left',
            4: 'lying down on right',
            5: 'lying down on stomach',
            6: 'miscellaneous movements',
            7: 'normal walking',
            8: 'running',
            9: 'shuffle walking',
            10: 'sitting/standing'
        }

    case 21:
        result = integer_mapping == {
            0: 'lying down on back',
            1: 'lying down on left',
            2: 'lying down on right',
            3: 'lying down on stomach',
            4: 'sitting/standing'
        }

    case 22:
        result = integer_mapping == {
            0: 'coughing',
            1: 'hyperventilating',
            2: 'normal',
        }

    case 31:
        result = integer_mapping == {
            0: 'lying down on back',
            1: 'lying down on left',
            2: 'lying down on right',
            3: 'lying down on stomach',
            4: 'sitting/standing'
        }

    case 32:
        result = integer_mapping == {
            0: 'coughing',
            1: 'hyperventilating',
            2: 'normal',
            3: 'other'
        }


    case 4:
        result = integer_mapping == {
            0: 'lying down on back coughing',
            1: 'lying down on back hyperventilating',
            2: 'lying down on back normal',
            3: 'lying down on back other',
            4: 'lying down on left coughing',
            5: 'lying down on left hyperventilating',
            6: 'lying down on left normal',
            7: 'lying down on left other',
            8: 'lying down on right coughing',
            9: 'lying down on right hyperventilating',
            10: 'lying down on right normal',
            11: 'lying down on right other',
            12: 'lying down on stomach coughing',
            13: 'lying down on stomach hyperventilating',
            14: 'lying down on stomach normal',
            15: 'lying down on stomach other',
            16: 'sitting/standing coughing',
            17: 'sitting/standing hyperventilating',
            18: 'sitting/standing normal',
            19: 'sitting/standing other'
        }

    case 5:
        result = integer_mapping == {
            0: 'ascending stairs normal',
            1: 'descending stairs normal',
            2: 'lying down on back coughing',
            3: 'lying down on back hyperventilating',
            4: 'lying down on back laughing',
            5: 'lying down on back normal',
            6: 'lying down on back singing',
            7: 'lying down on back talking',
            8: 'lying down on left coughing',
            9: 'lying down on left hyperventilating',
            10: 'lying down on left laughing',
            11: 'lying down on left normal',
            12: 'lying down on left singing',
            13: 'lying down on left talking',
            14: 'lying down on right coughing',
            15: 'lying down on right hyperventilating',
            16: 'lying down on right laughing',
            17: 'lying down on right normal',
            18: 'lying down on right singing',
            19: 'lying down on right talking',
            20: 'lying down on stomach coughing',
            21: 'lying down on stomach hyperventilating',
            22: 'lying down on stomach laughing',
            23: 'lying down on stomach normal',
            24: 'lying down on stomach singing',
            25: 'lying down on stomach talking',
            26: 'miscellaneous movements normal',
            27: 'normal walking normal',
            28: 'running normal',
            29: 'shuffle walking normal',
            30: 'sitting/standing coughing',
            31: 'sitting/standing eating',
            32: 'sitting/standing hyperventilating',
            33: 'sitting/standing laughing',
            34: 'sitting/standing normal',
            35: 'sitting/standing singing',
            36: 'sitting/standing talking'
        }
    case _:
        print(integer_mapping)
result

True

In [8]:
import tensorflow as tf
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

['/device:GPU:0']

In [9]:
!pip install kapre

In [10]:
# import numpy as np
# from tensorflow.keras import backend as K
# from tensorflow.python.keras.layers import InputSpec, Layer

# class Argmax(Layer):
#     """
#     Based on https://github.com/YerevaNN/R-NET-in-Keras/blob/master/layers/Argmax.py
#     """
#     def __init__(self, axis=-1, **kwargs):
#         super(Argmax, self).__init__(**kwargs)
#         self.supports_masking = True
#         self.axis = axis

#     def call(self, inputs, mask=None):
#         return K.argmax(inputs, axis=self.axis)

#     def compute_output_shape(self, input_shape):
#         input_shape = np.array(input_shape)
#         del input_shape[self.axis]
#         return tuple(input_shape)

#     def compute_mask(self, x, mask):
#         return None

#     def get_config(self):
#         config = {'axis': self.axis}
#         base_config = super(Argmax, self).get_config()
#         return dict(np.array(base_config.items()) + np.array(config.items()))

In [11]:
from tensorflow.keras.models import Sequential
from kapre import STFT, Magnitude, MagnitudeToDecibel
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Softmax, Reshape, Conv2D, Flatten
from tensorflow.keras.models import load_model
from keras.regularizers import l1_l2



# Load the model

if os.path.exists(model_path):
    model = load_model(model_path)
else:
    # Define the input shape for your audio data (e.g., 1D audio signal with a certain sampling rate)
    input_shape = (X_train.shape[1], X_train.shape[2])  # Replace with your actual input shape

    model = Sequential()

    # A STFT layer
    model.add(STFT(n_fft=128, win_length=5, hop_length=1,
                window_name=None, pad_end=True,
                input_data_format='channels_last', output_data_format='channels_last',
                input_shape=input_shape))
    model.add(Magnitude())
    model.add(MagnitudeToDecibel())  # these three layers can be replaced with get_stft_magnitude_layer()
    # Alternatively, you may want to use a melspectrogram layer
    # melgram_layer = get_melspectrogram_layer()
    # or log-frequency layer
    # log_stft_layer = get_log_frequency_spectrogram_layer()

    model.add(Conv2D(16, 5, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Conv2D(32, 5, activation='relu')) # , padding="same")
    model.add(Dropout(0.2))

    # model.add(Conv2D(64, 3, activation='relu', padding="same"))
    # model.add(Dropout(0.2))

    model.add(Conv2D(32, 5, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Conv2D(16, 5, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Flatten())

    # model.add(Reshape((window_size, 65 * 32), input_shape=(-1, window_size, 65, 32)))

    # # First LSTM layer with Dropout
    # model.add(LSTM(256, input_shape = (window_size, 65 * 32), return_sequences=True))
    # # model.add(LSTM(256, input_shape = input_shape, return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(BatchNormalization())

    # # Second LSTM layer with Dropout
    # model.add(LSTM(256, return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(BatchNormalization())

    # # Third LSTM layer with Dropout
    # model.add(LSTM(256))
    # model.add(Dropout(0.2))
    # model.add(BatchNormalization())

    # Fully connected layers
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    # Fully connected layers
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(y_train.shape[1], activation='softmax'))


    # # add more layers as you want
    # model.add(Conv2D(128, (3, 3), strides=(2, 2)))
    # model.add(BatchNormalization())
    # model.add(ReLU())
    # model.add(GlobalAveragePooling2D())

    # model.add(Dense(y_train.shape[1]))
    # model.add(Softmax())

    # Compile the model

    model.summary()

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 stft (STFT)                 (None, 50, 65, 3)         0         
                                                                 
 magnitude (Magnitude)       (None, 50, 65, 3)         0         
                                                                 
 magnitude_to_decibel (Magn  (None, 50, 65, 3)         0         
 itudeToDecibel)                                                 
                                                                 
 conv2d (Conv2D)             (None, 46, 61, 16)        1216      
                                                                 
 dropout (Dropout)           (None, 46, 61, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 57, 32)        12832     
                                                        

In [12]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

3908/3908 [==============================] - 36s 7ms/step - loss: 2.3944 - accuracy: 0.1008
Test Loss: 2.3944480419158936, Test Accuracy: 0.10077091306447983


In [ ]:
from keras.callbacks import EarlyStopping

if tainable:
    # Train the model with early stopping
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=512*4, callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=10, baseline=0.85, restore_best_weights=True),
        # EarlyStopping(monitor='loss', patience=5, baseline=0.1)
    ])
    # model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=512, callbacks=[
    #     EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    #     # EarlyStopping(monitor='loss', patience=5, baseline=0.1)
    # ])
    # model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[
    #     EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    #     # EarlyStopping(monitor='loss', patience=5, baseline=0.1)
    # ])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/100
205/205 [==============================] - 156s 650ms/step - loss: 1.8427 - accuracy: 0.2783 - val_loss: 1.4703 - val_accuracy: 0.3531
Epoch 2/100
205/205 [==============================] - 130s 634ms/step - loss: 1.4340 - accuracy: 0.3917 - val_loss: 1.2336 - val_accuracy: 0.4641
Epoch 3/100
205/205 [==============================] - 129s 630ms/step - loss: 1.3340 - accuracy: 0.4668 - val_loss: 1.1903 - val_accuracy: 0.5154
Epoch 4/100
205/205 [==============================] - 128s 627ms/step - loss: 1.1913 - accuracy: 0.5288 - val_loss: 0.9787 - val_accuracy: 0.6168
Epoch 5/100
205/205 [==============================] - 127s 620ms/step - loss: 1.0562 - accuracy: 0.5811 - val_loss: 1.0095 - val_accuracy: 0.6008
Epoch 6/100
205/205 [==============================] - 127s 620ms/step - loss: 0.9974 - accuracy: 0.6043 - val_loss: 0.9767 - val_accuracy: 0.6016
Epoch 7/100
205/205 [==============================] - 127s 620ms/step - loss: 0.9452 - accuracy: 0.6246 - val_loss: 1

In [ ]:
from kapre import STFTTflite, MagnitudeTflite

if True:
    model.save(model_path)

    model_tflite = Sequential()

    model_tflite.add(STFTTflite(n_fft=128, win_length=5, hop_length=1,
                window_name=None, pad_end=True,
                input_data_format='channels_last', output_data_format='channels_last',
                input_shape=input_shape))
    model_tflite.add(MagnitudeTflite())
    model_tflite.add(MagnitudeToDecibel())

    model_tflite.add(Conv2D(16, 5, activation='relu'))
    model_tflite.add(Dropout(0.2))

    model_tflite.add(Conv2D(32, 5, activation='relu'))
    model_tflite.add(Dropout(0.2))

    # model_tflite.add(Conv2D(64, 3, activation='relu', padding="same"))
    # model_tflite.add(Dropout(0.2))

    model_tflite.add(Conv2D(32, 5, activation='relu'))
    model_tflite.add(Dropout(0.2))

    model_tflite.add(Conv2D(16, 5, activation='relu'))
    model_tflite.add(Dropout(0.2))

    model_tflite.add(Flatten())

    # model_tflite.add(Reshape((window_size, 65 * 32), input_shape=(-1, window_size, 65, 32)))

    # # First LSTM layer with Dropout
    # model_tflite.add(LSTM(256, input_shape = (50, 65 * 32), return_sequences=True))
    # model_tflite.add(Dropout(0.2))
    # model_tflite.add(BatchNormalization())

    # # Second LSTM layer with Dropout
    # model_tflite.add(LSTM(256, return_sequences=True))
    # model_tflite.add(Dropout(0.2))
    # model_tflite.add(BatchNormalization())

    # # Third LSTM layer with Dropout
    # model_tflite.add(LSTM(256))
    # model_tflite.add(Dropout(0.2))
    # model_tflite.add(BatchNormalization())

    # Fully connected layers
    model_tflite.add(Dense(32, activation='relu'))
    model_tflite.add(Dropout(0.2))

    # Fully connected layers
    model_tflite.add(Dense(16, activation='relu'))
    model_tflite.add(Dropout(0.2))

    model_tflite.add(Dense(y_train.shape[1], activation='softmax'))

    # load the trained weights into the tflite compatible model.
    model_tflite.set_weights(model.get_weights())

    # # Save the entire model to a HDF5 file
    # run_model = tf.function(lambda x: model_tflite(x))
    # # This is important, let's fix the input size.
    # concrete_func = run_model.get_concrete_function(
    #     tf.TensorSpec((1, X_train.shape[1], X_train.shape[2]), model_tflite.inputs[0].dtype))

    # # model directory.
    # model_tflite.save('temp', save_format="tf", signatures=concrete_func)
    # converter = tf.lite.TFLiteConverter.from_saved_model('temp')

    # Convert the model.
    converter = tf.lite.TFLiteConverter.from_keras_model(model_tflite)

    model_tflite_path = '.'.join([model_path.split('.')[0], 'tflite'])
    tflite_model = converter.convert()
    open(model_tflite_path, "wb").write(tflite_model)

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from scipy.signal import spectrogram

# Load the model
if os.path.exists(model_path):
    model = load_model(model_path)

# Read and preprocess the new CSV file
# new_csv_file = "/content/drive/MyDrive/Colab Notebooks/pdiot-ml/pdiot-data/test_data/Respeck_s1911593_Sitting_Normal_clean_27-09-2023_14-00-59.csv"
# new_csv_file = "/content/drive/MyDrive/Colab Notebooks/pdiot-ml/pdiot-data/updated_anonymized_dataset_2023/Respeck/s100/s100_respeck_ascending_breathingNormal.csv"
# new_csv_file = "/content/drive/MyDrive/Colab Notebooks/pdiot-ml/pdiot-data/updated_anonymized_dataset_2023/Respeck/s100/s100_respeck_sitting_breathingNormal.csv"
new_csv_file = "/content/drive/MyDrive/Colab Notebooks/pdiot-ml/pdiot-data/updated_anonymized_dataset_2023/Respeck/s100/s100_respeck_sitting_coughing.csv"
# # new_csv_file = ""


match task_index:
    case 4:
        new_df = pd.read_csv(new_csv_file)[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
    case 5:
        new_df = pd.read_csv(new_csv_file)[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
    case _:
        new_df = pd.read_csv(new_csv_file)[['accel_x', 'accel_y', 'accel_z']]


match task_index:
    case 4:
        # norm the gyro data
        for col in ['gyro_x', 'gyro_y', 'gyro_z']:
            # https://pdf1.alldatasheet.com/datasheet-pdf/view/678850/AD/ADXRS300_15.html
            min_val = -300
            max_val = 300

            new_df[col] = (new_df[col] - min_val) / (max_val - min_val)
    case 5:
        # norm the gyro data
        for col in ['gyro_x', 'gyro_y', 'gyro_z']:
            # https://pdf1.alldatasheet.com/datasheet-pdf/view/678850/AD/ADXRS300_15.html
            min_val = -300
            max_val = 300

            new_df[col] = (new_df[col] - min_val) / (max_val - min_val)

# Window the new sequence
window_size = 50  # Define the size of the window
stride = 1  # Define the stride of the window

new_windows = []
new_sequence = new_df.values
for j in range(0, len(new_sequence) - window_size, stride):
    window = new_sequence[j:j + window_size]

    # Compute the spectrogram of the window
    # _, _, Sxx = spectrogram(window, axis=0)

    # new_windows.append(Sxx)
    new_windows.append(window)

new_windows = np.array(new_windows)

# Make predictions
predicted_label_indices = model.predict(new_windows)
# predicted_label_indices = np.argmax(predictions, axis=1)

print(predicted_label_indices)

# Assuming label_encoder is already fitted on the labels during training
predicted_labels = label_encoder.inverse_transform(predicted_label_indices)

# # Choose the most frequent label as the final prediction using NumPy
# unique_labels, counts = np.unique(predicted_labels, return_counts=True)
# final_prediction = unique_labels[np.argmax(counts)]

print(f"The predicted label for the new data is: {predicted_labels}")



In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming the TensorFlow model is stored in a variable called 'model'
# Replace 'model' with your actual model variable if different

# Generating the model architecture diagram and saving it as an image
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
print(model.summary())
